In [1]:
import os
import pandas as pd

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show, save
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.palettes import Spectral11, colorblind, Inferno, BuGn, brewer
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource,LinearColorMapper,BasicTicker, PrintfTickFormatter, ColorBar
import datetime
import os

In [3]:
folders = [f.name for f in os.scandir('.') if f.is_dir()][1:]

In [4]:
print(folders)

['001', '003', '004', '005', '006', '007', '008', '009', '010', '011', '013', '015', '016', '017', '019', '021', '024', '025', '026', '027', '028', '029', '030', '031', '032', '034', '037', '038', '039', '040', '041', '042', '044', '045', '047', '048', '049', '050', '052', '053', '055', '058', '059', '063', '065', '066', '068', '101', '102', '104', '105', '106', '107', '108', '109', '110', '112', '113', '114', '115', '116', '117', '121', '122', '123', '124', '125', '127', '128', '129', '130', '201', '202', '203', '204', '205', '206', '207', '208', '301_OFF', '301_ON', '302_OFF', '302_ON', '303', '304_OFF', '304_ON', '305']


In [5]:
top_home_page = """
<!DOCTYPE html>
<html>
	<head>
		<title>Home Survey</title>
		<style type="text/css">
			ul{
				float:left;
				list-style: none;
			}
			body{
				font-size: 20px;
				font-family: Open Sans;
				line-height: 32px;
				text-align: center;
			}
			a{
				text-decoration: none;
				display: block;
				padding: 10px 38px 10px 38px;
				border: 1px solid grey;
				background-color: lightgrey;
			}
			a:hover{
				background-color: navy;
				color: lightgrey;
			}
		</style>
	</head>
	<body>
    <ul>
        <li class="fir"><strong>Florida</strong></li>
"""
bottom_home_page = """
	</body>
</html>
"""

In [6]:
#Create a list of houses
def createList(folders,borneInf,borneSup):
    liste = []
    for folder in folders :
        if (int(folder[0:3])>= borneInf) and (int(folder[0:3])<borneSup):
            liste +=[folder]
    return liste       

In [7]:
sw0 = createList(folders,0,200)
#sw1 = createList(folders,100,200)
sw2 = createList(folders,200,300)
sw3 = createList(folders,300,400)
sw4 = createList(folders,400,500)

for f in sw4:
    top_home_page += """<li><a href='"""+f+"""/index.html'>"""+f+"""</a></li>"""
top_home_page += """
</ul>
    <ul>
        <li class="fir"><strong>Portland</strong></li>"""
for f in sw3:
    top_home_page += """<li><a href='"""+f+"""/index.html'>"""+f+"""</a></li>"""
top_home_page += """
</ul>
    <ul>
        <li class="fir"><strong>Colorado</strong></li>"""
for f in sw2:
    top_home_page += """<li><a href='"""+f+"""/index.html'>"""+f+"""</a></li>"""
top_home_page += """
</ul>
    <ul>
        <li class="fir"><strong>California (HENGH)</strong></li>"""
for f in sw0:
    top_home_page += """<li><a href='"""+f+"""/index.html'>"""+f+"""</a></li>"""
top_home_page += bottom_home_page

In [8]:
#writes to html file
hp = open('index.html','w')
hp.write(top_home_page)
hp.close()

In [9]:
#Sort columns of the DataTable
def sortDataTable(data):
    data['Time'] = pd.to_datetime(data['Time'], format='%m/%d/%Y%H%M')
    time = data["Time"].values
    forms = []
    pms = []
    intemps = []
    outtemps = []
    pm25s = []
    pm10s = []
    co2s = []
    inrhs = []
    outrhs = []
    ibus = []
    anms = []
    mtrs = []
    doors = []
    title_array = []
    #iterate each column (column name) in dataset
    for x in data[1:]:
        if x[0:3] == "FRM":
            forms += [x]
        elif x == 'PMin' or x == 'PMout':
            pms += [x]
        elif x == 'OUT_T' or x == 'OUT2_T':
            outtemps += [x]
        elif x[0:2] == 'T_' or x[len(x)-2:] == '_T':
            intemps += [x]
        elif x == 'OUT_RH' or x == 'OUT2_RH':
            outrhs += [x]
        elif x[0:3] == 'RH_' or x[len(x)-3:] == '_RH':
            inrhs += [x]
        elif x[len(x)-5:] == '_PM10':
            pm10s += [x]
        elif x[len(x)-5:] == '_PM25':
            pm25s += [x]
        elif x[0:4] == 'CO2_' or x[len(x)-4:] == '_CO2' :
            co2s += [x]
        elif x[0:4] == 'IBU_':
            ibus += [x]
        elif x[0:4] == 'ANM_':
            anms += [x]
        elif x[0:4] == 'MTR_':
            mtrs += [x]
        elif x[0:4] == 'STA_':
            doors += [x]

    return (time, forms,pms,intemps,outtemps,pm25s,pm10s,co2s,inrhs,outrhs,ibus,anms,mtrs,doors)

In [14]:
#Create Summary Statistic and Create Bokeh Plots
def createSummary(liste,titre,graphs):
    output_file(folder+"/"+str(liste)+".html")
    plo = figure(title=titre, x_axis_label='Time', y_axis_label='ug/m3', x_axis_type='datetime')
    for t in range(0, len(liste)):
        plo.line(time, df[liste[t]].values, legend=liste[t], line_width=2, color =colarr[t])
        graphs+="""
        <p><b>"""+ liste[t] +"""<br></b>
        Mean: """+ str(round(df.loc[:,liste[t]].mean(),1))  + """<br>
        Minimum: """ + str(round(df.loc[:,liste[t]].min(),1)) + """<br>
        5th Percentile: """ + str(round(df.loc[:,liste[t]].quantile(q=0.05),1)) + """<br>
        25th Percentile: """ + str(round(df.loc[:,liste[t]].quantile(q=0.25),1)) + """<br>
        Median: """ + str(round(df.loc[:,liste[t]].quantile(q=0.50),1)) + """<br>
        75th Percentile: """ +str(round(df.loc[:,liste[t]].quantile(q=0.75),1))  + """<br>
        95th Percentile: """ + str(round(df.loc[:,liste[t]].quantile(q=0.95),1)) + """<br>
        Maximum: """ + str(round(df.loc[:,liste[t]].max(),1)) + """<br></p>
        """
    graphs+="</div>"
    save(plo)

In [17]:
# accessing the CSV dataset
colarr = ['red', 'blue', 'black', 'orange', 'purple', 'grey', 'green', 'pink', 'yellow', 'maroon', 'brown', 'navy', 'magenta', 'tan']
for folder in folders[0:5]:
    print('debut ' + folder)
    df = data = pd.read_csv(folder+"/DataTable"+folder+".csv", parse_dates=['Time'])
    time, forms,pms,intemps,outtemps,pm25s,pm10s,co2s,inrhs,outrhs,ibus,anms,mtrs,doors = sortDataTable(data)
    tophtml = """<!DOCTYPE html>
    <html>
    <head>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="stylesheet" href="https://cdn.pydata.org/bokeh/release/bokeh-0.12.10.min.css" type="text/css" />
    <script type="text/javascript" src="https://cdn.pydata.org/bokeh/release/bokeh-0.12.10.min.js"></script>
    <script type="text/javascript">
        Bokeh.set_log_level("info");
    </script>
    <title>"""+folder+"""</title>
    <style>
    * {box-sizing: border-box}
    body {font-family: "Lato", sans-serif;}
    /* Style the tab */
    .tab {
      float: left;
      border: 1px solid #ccc;
      background-color: #f1f1f1;
      width: 30%;
      height: 300px;
    }
    /* Style the buttons inside the tab */
    .tab button {
      display: block;
      background-color: inherit;
      color: black;
      padding: 22px 16px;
      width: 100%;
      border: none;
      outline: none;
      text-align: left;
      cursor: pointer;
      transition: 0.3s;
      font-size: 17px;
    }
    /* Change background color of buttons on hover */
    .tab button:hover {
      background-color: #ddd;
    }
    th,td{
        padding: 10px 20px 10px 20px;
        border: 1px solid darkgrey;
    }
    /* Create an active/current "tab button" class */
    .tab button.active {
      background-color: #ccc;
    }
    /* Style the tab content */
    .tabcontent {
      float: left;
      padding: 0px 12px;
      /*border: 1px solid #ccc;*/
      width: 70%;
      border-left: none;
      height: 300px;
    }
    iframe{
      /*margin-left:200px;*/
      width:700px;
      height:620px;
    }
    </style>
    </head>
    <body>
    <h2></h2>
    <div class="tab">
      <button class="tablinks" onclick="openCity(event, 'pms')" id="defaultOpen">MetOne PM</button>"""
    navlist = ""
    graphs = """
    <div id="pms" class="tabcontent"><h3>MetOne PM</h3><iframe src="pm.html"></iframe>"""
    if pms:
        createSummary(pms,"MetOnePM",graphs)
    if pm25s:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'pm25s')">AirVisual Pro PM2.5</button>"""
        graphs += """
        <div id="pm25s" class="tabcontent"><h3>AirVisual Pro PM2.5</h3><iframe src="pm25.html"></iframe>"""
        createSummary(pm25s,"AirVisual Pro PM2.5",graphs)
    if pm10s:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'pm10s')">AirVisual Pro PM10</button>"""
        graphs += """
        <div id="pm10s" class="tabcontent"><h3>AirVisual Pro PM10</h3><iframe src="pm10.html"></iframe>"""
        createSummary(pm10s,"AirVisual Pro PM10",graphs)
    if co2s:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'co2s')">CO2</button>"""
        graphs += """
        <div id="co2s" class="tabcontent"><h3>CO2</h3><iframe src="co2.html"></iframe>"""
        createSummary(co2s,"CO2",graphs)
    if inrhs:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'inrhs')">Indoor Relative Humidity</button>"""
        graphs += """
        <div id="inrhs" class="tabcontent"><h3>Indoor Relative Humidity</h3><iframe src="inrh.html"></iframe>"""
        createSummary(inrhs,"Indoor Relative Humidity",graphs)
    if outrhs:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'outrhs')">Outdoor Relative Humidity</button>"""
        graphs += """
        <div id="outrhs" class="tabcontent"><h3>Outdoor Relative Humidity</h3><iframe src="outrh.html"></iframe>"""
        createSummary(outrhs,"Outdoor Relative Humidity",graphs)
    if forms:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'forms')">Formaldahyde</button>"""
        graphs += """
        <div id="forms" class="tabcontent"><h3>Formaldahyde</h3><iframe src="frm.html"></iframe>"""
        createSummary(forms,"Formaldahyde",graphs)
    if intemps:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'intemps')">Indoor Temperature</button>"""
        graphs += """
        <div id="intemps" class="tabcontent"><h3>Indoor Temperature</h3><iframe src="intemp.html"></iframe>"""
        createSummary(intemps,"Indoor Temperature",graphs)
    if outtemps:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'outtemps')">Outdoor Temperature</button>"""
        graphs += """
        <div id="outtemps" class="tabcontent"><h3>Outdoor Temperature</h3><iframe src="outtemp.html"></iframe>"""
        createSummary(outtemps,"Outdoor Temperature",graphs)
    if ibus:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'ibus')">iButton Temperature</button>"""
        graphs += """
        <div id="ibus" class="tabcontent"><h3>iButton Temperature</h3><iframe src="ibo.html"></iframe>"""
        createSummary(ibus,"iButton Temperature",graphs)
    if mtrs:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'mtrs')">Motor On-Off</button>"""
        graphs += """
        <div id="mtrs" class="tabcontent"><h3>Motor On-Off</h3><iframe src="mtr.html"></iframe>"""
        createSummary(mtrs,"Motor On-Off",graphs)
    if anms:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'anms')">Anemometer</button>"""
        graphs += """
        <div id="anms" class="tabcontent"><h3>Anemometer</h3><iframe src="anm.html"></iframe>"""
        createSummary(anms,"Anemometer",graphs)
    if doors:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'doors')">Door State</button>"""
        graphs += """
        <div id="doors" class="tabcontent"><h3>Door State</h3><iframe src="sta.html"></iframe>"""
        createSummary(doors,"Door State",graphs)
    tophtml = tophtml + navlist + "</div>" + graphs + """<script>
function openCity(evt, cityName) {
  var i, tabcontent, tablinks;
  tabcontent = document.getElementsByClassName("tabcontent");
  for (i = 0; i < tabcontent.length; i++) {
    tabcontent[i].style.display = "none";
  }
  tablinks = document.getElementsByClassName("tablinks");
  for (i = 0; i < tablinks.length; i++) {
    tablinks[i].className = tablinks[i].className.replace(" active", "");
  }
  document.getElementById(cityName).style.display = "block";
  evt.currentTarget.className += " active";
}
// Get the element with id="defaultOpen" and click on it
document.getElementById("defaultOpen").click();
</script>
</body>
</html> """
#     print(tophtml)
    hp = open(folder+'/index.html','w')
    hp.write(tophtml)
    hp.close()

debut 001
debut 003
debut 004
debut 005
debut 006


In [18]:
createSummary(co2s,"CO2",graphs)